In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

movies = pd.read_csv('movies.dat',sep = '::',header = None,names = ['MovieID','Title','Genres'])
print(movies)


ratings = pd.read_csv('ratings.dat',sep = '::',header = None,names = ['UserID','MovieID','Rating','Timestamp'])
print(ratings)


users = pd.read_csv('users.dat',sep = '::',header = None,names = ['UserID','Gender','Age','Occupation','Zip-code'])

print(users)

<ipython-input-2-0f694b8aed6e>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv('movies.dat',sep = '::',header = None,names = ['MovieID','Title','Genres'])
<ipython-input-2-0f694b8aed6e>:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('ratings.dat',sep = '::',header = None,names = ['UserID','MovieID','Rating','Timestamp'])


      MovieID                               Title  \
0           1                    Toy Story (1995)   
1           2                      Jumanji (1995)   
2           3             Grumpier Old Men (1995)   
3           4            Waiting to Exhale (1995)   
4           5  Father of the Bride Part II (1995)   
...       ...                                 ...   
3878     3948             Meet the Parents (2000)   
3879     3949          Requiem for a Dream (2000)   
3880     3950                    Tigerland (2000)   
3881     3951             Two Family House (2000)   
3882     3952               Contender, The (2000)   

                            Genres  
0      Animation|Children's|Comedy  
1     Adventure|Children's|Fantasy  
2                   Comedy|Romance  
3                     Comedy|Drama  
4                           Comedy  
...                            ...  
3878                        Comedy  
3879                         Drama  
3880                         D

<ipython-input-2-0f694b8aed6e>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_csv('users.dat',sep = '::',header = None,names = ['UserID','Gender','Age','Occupation','Zip-code'])


In [3]:
ratings['MovieID'].max()

3952

In [4]:
ratings_mat = np.ndarray(shape=(np.max(ratings.MovieID.values), np.max(ratings.UserID.values)),dtype=np.uint8)
ratings_mat[ratings.MovieID.values-1, ratings.UserID.values-1] = ratings.Rating.values

normalised_mat = ratings_mat - np.asarray([(np.mean(ratings_mat, 1))]).T
A = normalised_mat.T / np.sqrt(ratings_mat.shape[0] - 1)

In [34]:
U, S, V = np.linalg.svd(A)

In [35]:
V.T.shape

(3952, 3952)

In [36]:
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
def top_cosine_similarity(data, movie_id, top_n=10):
    index = movie_id - 1 # Movie id starts from 1 in the dataset
    movie_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data)) #array
    similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude) #array
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

def print_similar_movies(movie_data, movie_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    movie_data[movie_data.MovieID == movie_id].Title.values[0]))
    for id in top_indexes + 1:
        print(movie_data[movie_data.MovieID == id].Title.values[0])
        print(movie_data[movie_data.MovieID == id].Genres.values[0])

In [39]:
k = 50
MovieID = 1 # (getting an id from movies.dat)
top_n = 10
sliced = V.T[:, :k] # representative data
indexes = top_cosine_similarity(sliced, MovieID, top_n)
indexes

<ipython-input-36-a64d5391929f>:7: RuntimeWarning: invalid value encountered in true_divide
  similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude) #array


array([   0, 3113,   33, 2354, 2320, 2383,  587, 3156,  530, 2686])

In [40]:
sliced.shape

(3952, 50)

In [41]:
#movie recommendations using SVD generated vectors
print_similar_movies(movies, MovieID, indexes)

Recommendations for Toy Story (1995): 

Toy Story (1995)
Animation|Children's|Comedy
Toy Story 2 (1999)
Animation|Children's|Comedy
Babe (1995)
Children's|Comedy|Drama
Bug's Life, A (1998)
Animation|Children's|Comedy
Pleasantville (1998)
Comedy
Babe: Pig in the City (1998)
Children's|Comedy
Aladdin (1992)
Animation|Children's|Comedy|Musical
Stuart Little (1999)
Children's|Comedy
Secret Garden, The (1993)
Children's|Drama
Tarzan (1999)
Animation|Children's


In [18]:
ratings_mat.shape

(3952, 6040)

In [19]:
norm = np.linalg.norm(ratings_mat)
normal_array = ratings_mat/norm

In [21]:
from numpy import mean
from numpy import cov
from numpy.linalg import eig
from numpy import array

# calculate covariance matrix of normalized matrix
V = cov(normal_array)
# eigendecomposition of covariance matrix
values, vectors = eig(V)

In [22]:
vectors.shape

(3952, 3952)

In [42]:
k = 50
MovieID = 1 # (getting an id from movies)
top_n = 10
sliced = vectors[:, :k]
indexes = top_cosine_similarity(sliced, MovieID, top_n)
indexes

<ipython-input-36-a64d5391929f>:7: RuntimeWarning: invalid value encountered in true_divide
  similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude) #array


array([   0, 3113,   33, 2354, 2320, 2383,  587, 3156,  530, 2686])

In [43]:
#movie recommendations using PCA eigen vectors
print_similar_movies(movies, MovieID, indexes)

Recommendations for Toy Story (1995): 

Toy Story (1995)
Animation|Children's|Comedy
Toy Story 2 (1999)
Animation|Children's|Comedy
Babe (1995)
Children's|Comedy|Drama
Bug's Life, A (1998)
Animation|Children's|Comedy
Pleasantville (1998)
Comedy
Babe: Pig in the City (1998)
Children's|Comedy
Aladdin (1992)
Animation|Children's|Comedy|Musical
Stuart Little (1999)
Children's|Comedy
Secret Garden, The (1993)
Children's|Drama
Tarzan (1999)
Animation|Children's


Singular value decomposition (SVD) and principal component analysis (PCA) are two eigenvalue methods used to reduce a high-dimensional data set into fewer dimensions while retaining important information.

PCA: If your features are least sensitive (informative) towards the mean of the distribution, then it makes sense to subtract the mean. If the features are most sensitive towards the high values, then subtracting the mean does not make sense.

SVD: it does not subtract the means but often as a first step projects the data on the mean of all data points

conclusion:
movie recommendation system developed using both PCA and SVD gave same results